In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [ ]:
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", \
           "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", \
           "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

r = requests.get('https://www.imapi.org/perfil/santo_andre-sp', headers=headers)#, proxies=proxies)
content = r.content
soup = BeautifulSoup(content)

In [ ]:
labels=[]
values=[]
dims=[]
for d in soup.findAll('article', attrs={'class':'dimensao-sticky'}):
  for d1 in d.findAll('div', attrs={'class':'col-xs-3'}):
      label =  d1.find('p', attrs={'class':'label'})
      print('Label:', label.text)
      dado = d1.find('p', attrs={'class':'dado'})
      print('Dado:', dado.text)
      print('---')

for d in soup.findAll('g'):
  if d.has_attr('data-nota'):
    print('label: ', d['data-titulo'])
    print('valor: ', d['data-nota'])
    print('label: ', d['data-titulo'])
    print('valor: ', d['data-desempenho'])
    print('---')

for d in soup.findAll('section', attrs={'class':'row bottom-xs'}):
 for d1 in d.findAll('div', attrs={'class':'col-xs-4 tl'}): 
    dado = d1.find('p', attrs={'class':'dado'})
    print('Dado:', dado.text)
    label =  d1.find('p', attrs={'class':'label'})
    print('Label:', label.text) 



for s in soup.findAll('section', attrs={'class':'card'}):
  for d in s.findAll('article', attrs={'class':'indicador'}):
      dado = d.find('p', attrs={'class':'dado'})
      print('Seção', s['data-dimensao'])
      print('Dado:', dado.text)
      
      label =  d.find('p', attrs={'class':'label'})
      print('Label:', label.text) 
      municipio = d.find('div', attrs={'class':'local ativo'})
      if municipio != None and municipio.has_attr('data-nota'):
        print('Valor: ', municipio['data-nota'])
        valor = municipio['data-nota']
      else:
        print('Valor: N/A')
        valor = 'N/A'
      print('---')
      dims.append(s['data-dimensao'])
      labels.append(label.text)
      values.append(valor)

res = {'dimensao:':dims, 'label':labels, 'valor':values}

In [ ]:
pd.DataFrame().from_dict(res)

Dados dinâmicos (Javascript)

In [ ]:
!pip install selenium
!apt install chromium-chromedriver

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup

url='https://www.imapi.org/perfil/porto-alegre-rs'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=chrome_options)
driver =webdriver.Chrome('chromedriver',options=chrome_options)

driver.get(url)
html = driver.page_source
soup2 = BeautifulSoup(html, 'html.parser')

In [ ]:
out = [option.text for option in soup2.select('#municipios option')]

In [ ]:
municipios = out

In [ ]:
municipios

In [ ]:
from time import sleep
from random import randint

In [ ]:
labels=[]
values=[]
munis = []
estados = []
dims=[]
i =  1
### top 10 
for muni in municipios[:10]:
  #print('trying:','https://www.imapi.org/perfil/', muni )
  #print(i, '/', len(municipios))
  
  print(i, sep=' ', end=' ', flush=True) 
  i = i + 1
  r = requests.get('https://www.imapi.org/perfil/'+muni, headers=headers)#, proxies=proxies)
  content = r.content
  sleep(randint(1,2))
  soup = BeautifulSoup(content)
  for s in soup.findAll('section', attrs={'class':'card'}):
    for d in s.findAll('article', attrs={'class':'indicador'}):
        dado = d.find('p', attrs={'class':'dado'})
       # print('Seção', s['data-dimensao'])
       # print('Dado:', dado.text)
        
        label =  d.find('p', attrs={'class':'label'})
        #print('Label:', label.text) 
        municipio = d.find('div', attrs={'class':'local ativo'})
        if municipio != None and municipio.has_attr('data-nota'):
        #  print('Valor: ', municipio['data-nota'])
          valor = municipio['data-nota']
        else:
         # print('Valor: N/A')
          valor = np.NaN
        #print('---')
        munis.append(muni.split('-')[0])
        estados.append(muni.split('-')[1])
        dims.append(s['data-dimensao'])
        labels.append(label.text)
        values.append(valor)
      

  res = {'estado':estados, 'municipio':munis, 'dimensao':dims, 'label':labels, 'valor':values}
  res_df = pd.DataFrame().from_dict(res)
  res_df.to_csv('imapi.csv')

In [ ]:
t = pd.read_csv('imapi.csv', index_col=0)

In [ ]:
t